In [1]:
from google.colab import drive

PATH_DRIVE = '/content/drive'
drive.mount(PATH_DRIVE)

Mounted at /content/drive


In [2]:
import os
import pandas as pd

from datetime import datetime
from pathlib import Path

PATH_BASE = Path(PATH_DRIVE)/'MyDrive'
print(os.listdir(PATH_BASE))

['문서', '사진', '설문', 'drawio', 'Colab Notebooks', 'data', 'models']


In [3]:
! pip3 install -U datasets transformers evaluate sentencepiece

import torch
import datasets

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from evaluate import load

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
now = datetime.now()
timestamp = datetime.timestamp(now)

NAME_MODEL = 'cross-encoder/nli-deberta-v3-base'
NAME_DIR = f'mednli_deberta_{int(timestamp)}'

PATH_MODELS = PATH_BASE/'models'/NAME_DIR

dataset = datasets.load_dataset(
    'bigbio/mednli', 
    'mednli_source',
    data_dir=PATH_BASE/'data'
)
model = AutoModelForSequenceClassification.from_pretrained(NAME_MODEL)
tokenizer = AutoTokenizer.from_pretrained(NAME_MODEL)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset mednli downloaded and prepared to /root/.cache/huggingface/datasets/bigbio___mednli/mednli_source-61f28563cb5c4805/1.0.0/f18e31046c4bf68bbdc505fbf3dab924a4ea60082a7f41346e3a84b54f453535. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
def label_encode(data):
    labels = {'contradiction': 0, 'entailment': 1, 'neutral': 2}
    key = data['gold_label']
    data = {'label': labels[key]}

    return data


def tokenize(data):
    data = tokenizer(
        data['sentence1'],
        data['sentence2'],
        max_length=model.config.max_position_embeddings,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    return data


def compute_metrics(y):
    logits, labels = y
    preds = logits.argmax(axis=1)
    metrics = 'precision', 'recall', 'f1'
    result = {}

    result.update(
        load('accuracy').compute(
            predictions=preds,
            references=labels
        )
    )

    for metric in metrics:
        result.update(
            load(metric).compute(
                predictions=preds,
                references=labels,
                average='macro'
            )
        )

    return result


In [6]:
cols = [
    'input_ids', 
    'token_type_ids',
    'attention_mask', 
    'label'
]
cols = [col for col in dataset.column_names['test'] if col not in cols]

dataset = dataset.map(label_encode)
dataset = dataset.map(tokenize, batched=True)
dataset = dataset.remove_columns(cols)

  0%|          | 0/11232 [00:00<?, ?ex/s]

  0%|          | 0/1422 [00:00<?, ?ex/s]

  0%|          | 0/1395 [00:00<?, ?ex/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [7]:
numEpoch = 3
numBatch = 8
numStep = dataset.num_rows['train'] * numEpoch / numBatch

args = TrainingArguments(
    run_name=NAME_DIR,
    num_train_epochs=numEpoch,
    per_device_train_batch_size=numBatch,
    per_device_eval_batch_size=numBatch*4,
    learning_rate=2e-5,
    weight_decay=1e-3,
    warmup_ratio=1e-2,
    warmup_steps=int(numStep/15),
    save_total_limit=3,
    save_steps=100,
    logging_steps=50,
    save_strategy='steps',
    evaluation_strategy='steps',
    output_dir=PATH_MODELS/'weights',
    logging_dir=PATH_MODELS/'logs',
    seed=0,
    metric_for_best_model='f1',
    load_best_model_at_end=True
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [8]:
if not os.path.exists(PATH_MODELS):
    os.mkdir(PATH_MODELS)

with open(PATH_MODELS/'args.json', 'w', encoding='utf-8') as f:
    text = args.to_json_string()
    f.write(text)

trainer.train()
trainer.save_model(PATH_MODELS/'weights')

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11232
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4212
  Number of trainable parameters = 184424451


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,1.050000,0.724653,0.784229,0.785132,0.784229,0.783760
200,0.624800,0.503933,0.802151,0.808080,0.802151,0.804136
300,0.590500,0.514262,0.811470,0.811596,0.811470,0.811490
400,0.557300,0.480257,0.813620,0.812640,0.813620,0.812297
500,0.564600,0.442855,0.830108,0.840885,0.830108,0.830765
600,0.524300,0.463941,0.830824,0.832544,0.830824,0.831350
700,0.487700,0.457727,0.833692,0.833971,0.833692,0.833262
800,0.512800,0.417193,0.837993,0.842077,0.837993,0.839111
900,0.494400,0.393531,0.845161,0.846470,0.845161,0.845353
1000,0.406300,0.475387,0.847312,0.849171,0.847312,0.848043


***** Running Evaluation *****
  Num examples = 1395
  Batch size = 32


***** Running Evaluation *****
  Num examples = 1395
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/models/mednli_deberta_1675172952/weights/checkpoint-200
Configuration saved in /content/drive/MyDrive/models/mednli_deberta_1675172952/weights/checkpoint-200/config.json
Model weights saved in /content/drive/MyDrive/models/mednli_deberta_1675172952/weights/checkpoint-200/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1395
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1395
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/models/mednli_deberta_1675172952/weights/checkpoint-400
Configuration saved in /content/drive/MyDrive/models/mednli_deberta_1675172952/weights/checkpoint-400/config.json
Model weights saved in /content/drive/MyDrive/models/mednli_deberta_1675172952/weights/checkpoint-400/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1395
  Batch size = 32
***** Running Evaluation *****
  Num

In [9]:
result = trainer.evaluate(dataset['test'])
df = pd.DataFrame.from_dict([result])

df.to_csv(PATH_MODELS/'result.csv', index=False, encoding='utf-8')
df

***** Running Evaluation *****
  Num examples = 1422
  Batch size = 32


,eval_loss,eval_accuracy,eval_precision,eval_recall,eval_f1,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,0.440221,0.836146,0.83588,0.836146,0.835976,68.3004,20.82,0.659,1.28
